# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# PIPELINE EXTRAÇÃO

In [ ]:
from modules.youtube.extract.ytbpy import channel, playlist, utils, video
from modules import db_manager
from datetime import datetime
from jinja2 import Template
from tqdm import tqdm
import pandas as pd
import yt_dlp
import os
import time

def wait(seconds=60):
    for _ in range(seconds):
        time.sleep(1)

path_sql = "repository\\querys\\operation\\insert\\playlist.sql"
with open(path_sql, "r", encoding="utf-8") as file:
    sql_query_insert = file.read()

path_sql = "repository\\querys\\operation\\read\\channels.sql"
with open(path_sql, "r", encoding="utf-8") as file:
    sql_query_read = file.read()
df_channel = db_manager.read_df(sql_query_read)
    
with open("repository\\querys\\operation\\read\\playlist.sql", "r", encoding="utf-8") as file:
    sql_query_read = file.read()
    df_playlist = db_manager.read_df(sql_query_read)

In [ ]:
for name_handle in tqdm(df_channel['handle'], desc="Canais"):
    # Configurações para suprimir a saída verbose
    ydl_opts = {
        'skip_download': True, 
        'extract_flat': True,
        'quiet': True,           # Suprime a maioria das mensagens
        'no_warnings': True,     # Remove warnings
        'ignoreerrors': True     # Ignora erros não-críticos
    }
    ydl = yt_dlp.YoutubeDL(ydl_opts)
    info = ydl.extract_info(f"https://www.youtube.com/{name_handle}/playlists", download=False)
    if info:
        for playlist_info in tqdm(info.get('entries', []), desc=f"Playlists de {name_handle}", leave=False):
            cut = df_playlist[df_playlist.id_playlist.isin([playlist_info["id"]])]
            if cut.empty:
            
                channel_info        = channel.get_channel_info(f'https://www.youtube.com/{name_handle}')
                ID_YOUTUBE          = channel_info["channel_id"].replace("'", "")  # Escape single quotes for SQL
                
                # Validar e truncar strings conforme os limites do banco
                ID_PLAYLIST         = playlist_info['id'].replace("'", "").strip()          if playlist_info['id'] else ""  # Limitado a 30 chars
                ID_GAME             = 1  # Definindo um valor padrão ao invés de None
                TITLE               = playlist_info['title'].replace("'", "").strip()       if playlist_info['title'] else ""  # Limitado a 50 chars
                DESCRIPTION         = ""  # Limitado a 1000 chars (se aplicável)
                REGISTER_DATE       = datetime.now().strftime("%Y-%m-%d")  # Formato DATE correto
                
                # Debug: verificar tamanhos
                print(f"ID_YOUTUBE: {len(ID_YOUTUBE)} chars - {ID_YOUTUBE}")
                print(f"TITLE: {len(TITLE)} chars - {TITLE}")
                
                sql_template        = Template(sql_query_insert)
                rendered_sql        = sql_template.render(
                    ID_PLAYLIST     = ID_PLAYLIST,
                    ID_YOUTUBE      = ID_YOUTUBE,
                    ID_GAME         = ID_GAME,
                    TITLE           = TITLE,
                    DESCRIPTION     = DESCRIPTION,
                    REGISTER_DATE   = REGISTER_DATE,
                ).replace("'None'", "NULL").replace("None", "NULL")\
                .replace("'False'", "FALSE").replace("False", "FALSE")\
                .replace("'True'", "TRUE").replace("True", "TRUE")
                
                try:
                    db_manager.execute_raw(rendered_sql)
                except Exception as e:
                    print(f"Error inserting playlist data: {e}")
                    print(f"SQL: {rendered_sql}")
                    db_manager.connection.rollback()
